In [ ]:
# Iterative RAFFLE structure search
from ase.io import read
from raffle.generator import raffle_generator
from mace.calculators import mace_mp
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
generator = raffle_generator()
calc = mace_mp(model="medium", dispersion=False, default_dtype="float32", device='cpu')

In [ ]:
aa_stack = read("POSCAR_AA_stack")
ab_stack = read("POSCAR_AB_stack")
aabbcc_stack = read("POSCAR_AABBCC_stack")
aba_stack = read("POSCAR_ABA_stack")
lonsdaleite = read("POSCAR_lonsdaleite")

In [ ]:
aa_stack.calc = calc
ab_stack.calc = calc
aabbcc_stack.calc = calc
aba_stack.calc = calc
lonsdaleite.calc = calc

In [ ]:
# change aa_stack to same height as ab_stack
# this is to confirm that the differences in descriptor are not a result of different interlayer distances
ab_height = ab_stack.get_cell()[2, 2]
cell = aa_stack.get_cell()
aa_height = cell[2, 2]
cell[2, 2] = ab_height
aa_stack.set_cell(cell, scale_atoms=True)
aa_stack.calc = calc

In [ ]:
aa_descriptor = generator.distributions.generate_fingerprint(aa_stack)
ab_descriptor = generator.distributions.generate_fingerprint(ab_stack)
aba_descriptor = generator.distributions.generate_fingerprint(aba_stack)
aabbcc_descriptor = generator.distributions.generate_fingerprint(aabbcc_stack)
lonsdaleite_descriptor = generator.distributions.generate_fingerprint(lonsdaleite)

In [ ]:


fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot for each n-body function (2-body, 3-body, 4-body)
for j in range(3):
    # Calculate x-axis values
    x = np.arange(generator.distributions.cutoff_min[j],
                generator.distributions.cutoff_max[j] + generator.distributions.width[j],
                generator.distributions.width[j])

    # Plot on the respective subplot
    axes[j].plot(x, aa_descriptor[j], label='AA stack')
    axes[j].plot(x, ab_descriptor[j], label='AB stack')
    axes[j].plot(x, aba_descriptor[j], label='ABA stack')
    axes[j].plot(x, aabbcc_descriptor[j], label='AABBCC stack')
    axes[j].plot(x, lonsdaleite_descriptor[j], label='Lonsdaleite')

    # Set labels and title for each subplot
    axes[j].set_ylabel('Fingerprint value')
    axes[j].set_title(f'{j+2}-body fingerprint')
    axes[j].legend()

axes[0].set_xlabel('Distance (Å)')
axes[1].set_xlabel('3-body angle (radians)')
axes[2].set_xlabel('Improper dihedral angle (radians)')
plt.tight_layout()
plt.show()